In [1]:
import random
import numpy as np
from scipy.stats import t

In [2]:
#a simple function to sample from negative exponential
def neg_exp(lambd):
    return random.expovariate(lambd)

In [4]:
# define a helper function for the 95% confidence interval test
def CI95_upper(data):
    upper = np.mean(data) + 1.96*np.std(data)/np.sqrt(len(data))
    return upper

def CI95_lower(data):
    lower = np.mean(data) - 1.96*np.std(data)/np.sqrt(len(data))
    return lower

In [7]:
def dmm_queue(inter_arrival,m,iteration):
    
    avg_waiting_iter = []
    less_than45_iter = []
    
    for i in range(iteration):
        
        Customers = []

        # time in minute
        t = 0
        simulation_time = 60

        p = 25
        lambd = 1/p

        # initialize arrival date to be 0
        arrival_date = 0

        # set the number of servers
        servers = np.zeros(m)
        
        #The actual simulation happens here:
        while t<simulation_time:

            #the service time follows exponential distribution 
            service_time=neg_exp(lambd)

            #calculate arrival date and service time for new customer
            
            # the first m customers don't need to wait
            if len(Customers)+1<=m :
                arrival_date+=inter_arrival
                service_start_date=arrival_date
                service_end_date = service_start_date + service_time
                servers[len(Customers)] = service_end_date
            
            else:
                arrival_date+=inter_arrival
                
                # new customers who need to wait until the next avaialble clinician 
                if arrival_date <= min(servers):
                    service_start_date = min(servers)
                    service_end_date = service_start_date + service_time
                    servers[np.argmin(servers)] = min(servers) + service_time
                
                #new customers who don't need to wait  
                else: 
                    service_start_date = arrival_date
                    service_end_date = service_start_date + service_time
                    servers[np.argmin(servers)] = service_end_date

            #calculate the waiting time
            waiting_time = service_start_date - arrival_date

            cust_dict = {'arrival_date':arrival_date, 
                         'service_start_date': service_start_date,
                         'service_time':service_time,
                         'service_end_date':service_end_date,
                         'waiting_time':waiting_time}

            #create new customer
            Customers.append(cust_dict)

            #increment clock till next end of service
            t=arrival_date
        
        waiting_times = np.array([i['waiting_time'] for i in Customers])
        
        #calculate the average waiting time
        avg_waiting = np.mean(waiting_times)
        avg_waiting_iter.append(avg_waiting)
        
        #calculate the probability that the waiting time less than 45 minutes
        less_than45 = np.sum(waiting_times < 45) / len(waiting_times)
        less_than45_iter.append(less_than45)
        
    #compute the upper bound of 95% confidence interval for avg_waiting time
    if CI95_upper(avg_waiting_iter) <= 45:
        print('95% confident that the average waiting time is less than 45 minutes')
    else:
        print('failed')
    
    #compute the lower bound of 95% confidence interval for probability that waiting time is less than 45 minutes
    if CI95_lower(less_than45_iter) >= 0.8:
        print('95% confident that the probability that waiting time is less than 45 minutes is at least 80%') 
    else:
        print('failed')

In [34]:
dmm_queue(60/4.67,2,1000)

95% confident that the average waiting time is less than 45 minutes
95% confident that the probability that waiting time is less than 45 minutes is at least 80%
